In [115]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

%load_ext autoreload
%autoreload 2
import utils

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [116]:
#Extraigo datos:

df_reviews = pd.read_parquet('../data/user_reviews_limpo.parquet')
df_games = pd.read_parquet('../data/steam_games_limpo.parquet')
df_items = pd.read_parquet('../data/user_items_limpo.parquet')

## Análisis de sentimientos

Se pide crear una nueva columna llamada 'sentiment_analysis' que reemplace a 'reviews_review' donde se realice un análisis de sentimiento de los comentarios con la siguiente escala:

    *0 si es malo,
    *1 si es neutral o esta sin review
    *2 si es positivo.
    
Dado que el objetivo de este proyecto es realizar una prueba de concepto, consiguiendo un producto mínimo viable, se realiza un análisis de sentimiento básico utilizando TextBlob que es una biblioteca de procesamiento de lenguaje natural (NLP) en Python. El objetivo de esta metodología es asignar un valor numérico a un texto, en este caso a los comentarios que los usuarios dejaron para un juego determinado, para representar si el sentimiento expresado en el texto es negativo, neutral o positivo.

Esta metodología toma una revisión de texto como entrada, utiliza TextBlob para calcular la polaridad de sentimiento y luego clasifica la revisión como negativa, neutral o positiva en función de la polaridad calculada. En este caso, se consideraron las polaridades por defecto del modelo, el cuál utiliza umbrales -0.2 y 0.2, siendo polaridades negativas por debajo de -0.2, positivas por encima de 0.2 y neutrales entre medio de ambos.

In [117]:
df_reviews['sentiment_analysis'] = df_reviews['reviews_review'].apply(utils.sentiment_analysis)
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57367 entries, 0 to 57366
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             57367 non-null  object
 1   user_url            57367 non-null  object
 2   reviews_item_id     57367 non-null  int64 
 3   reviews_helpful     57367 non-null  object
 4   reviews_recommend   57367 non-null  bool  
 5   reviews_review      57367 non-null  object
 6   reviews_date        57367 non-null  object
 7   sentiment_analysis  57367 non-null  int64 
dtypes: bool(1), int64(2), object(5)
memory usage: 3.1+ MB


In [118]:
#Ya teniendo las reseñas como valor numerico, elimino la columna "reviews_review"
df_reviews = df_reviews.drop(columns=['reviews_review'])
df_reviews.columns

Index(['user_id', 'user_url', 'reviews_item_id', 'reviews_helpful',
       'reviews_recommend', 'reviews_date', 'sentiment_analysis'],
      dtype='object')

In [119]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094105 entries, 0 to 5094104
Data columns (total 7 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           int64 
 1   item_name         object
 2   playtime_forever  int64 
 3   steam_id          int64 
 4   items_count       int64 
 5   user_id           object
 6   user_url          object
dtypes: int64(4), object(3)
memory usage: 272.1+ MB


In [120]:
#renombre columna ID de games para posterior Merge
nuevos_nombres = {'id': 'item_id'}

df_games_rename = df_games.rename(columns=nuevos_nombres)
df_games_rename.head()

,genres,price,early_access,item_id,release_anio,publisher,app_name,title,developer
0,Action,4.99,0.0,761140.0,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Casual,4.99,0.0,761140.0,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Indie,4.99,0.0,761140.0,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Simulation,4.99,0.0,761140.0,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Strategy,4.99,0.0,761140.0,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro


In [121]:
df_games_rename['item_id'].astype(int)

0        761140
0        761140
0        761140
0        761140
0        761140
          ...  
32132    610660
32132    610660
32132    610660
32133    658870
32133    658870
Name: item_id, Length: 71551, dtype: int32

In [122]:
print(df_games_rename.info())

<class 'pandas.core.frame.DataFrame'>
Index: 71551 entries, 0 to 32133
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        71551 non-null  object 
 1   price         71551 non-null  float64
 2   early_access  71551 non-null  float64
 3   item_id       71551 non-null  float64
 4   release_anio  71498 non-null  object 
 5   publisher     71551 non-null  object 
 6   app_name      71551 non-null  object 
 7   title         71551 non-null  object 
 8   developer     71551 non-null  object 
dtypes: float64(3), object(6)
memory usage: 5.5+ MB
None


In [123]:
print(df_items.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094105 entries, 0 to 5094104
Data columns (total 7 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           int64 
 1   item_name         object
 2   playtime_forever  int64 
 3   steam_id          int64 
 4   items_count       int64 
 5   user_id           object
 6   user_url          object
dtypes: int64(4), object(3)
memory usage: 272.1+ MB
None


In [124]:
print (df_reviews.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57367 entries, 0 to 57366
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             57367 non-null  object
 1   user_url            57367 non-null  object
 2   reviews_item_id     57367 non-null  int64 
 3   reviews_helpful     57367 non-null  object
 4   reviews_recommend   57367 non-null  bool  
 5   reviews_date        57367 non-null  object
 6   sentiment_analysis  57367 non-null  int64 
dtypes: bool(1), int64(2), object(4)
memory usage: 2.7+ MB
None


In [125]:
#Modifico a formato Mmmmm Mmmmm 

df_games_rename['genres'] = df_games_rename['genres'].str.title()
df_items['item_name'] = df_items['item_name'].str.title()


In [126]:

# Eliminar duplicados en df_reviews
df_reviews = df_reviews.drop_duplicates(subset=['user_id'])

In [127]:
#Realizo primer merge
merged_df = pd.merge(df_reviews, df_items, how='inner', on='user_id')

In [128]:
#Realizo segundo merge
final_df = pd.merge(merged_df, df_games_rename, how='inner', on='item_id')

In [129]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5077395 entries, 0 to 5077394
Data columns (total 21 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   user_url_x          object 
 2   reviews_item_id     int64  
 3   reviews_helpful     object 
 4   reviews_recommend   bool   
 5   reviews_date        object 
 6   sentiment_analysis  int64  
 7   item_id             int64  
 8   item_name           object 
 9   playtime_forever    int64  
 10  steam_id            int64  
 11  items_count         int64  
 12  user_url_y          object 
 13  genres              object 
 14  price               float64
 15  early_access        float64
 16  release_anio        object 
 17  publisher           object 
 18  app_name            object 
 19  title               object 
 20  developer           object 
dtypes: bool(1), float64(2), int64(6), object(12)
memory usage: 779.6+ MB


In [130]:
utils.types_data_df(final_df)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,user_id,[<class 'str'>],100.00,0.00,0
1,user_url_x,[<class 'str'>],100.00,0.00,0
2,reviews_item_id,[<class 'int'>],100.00,0.00,0
3,reviews_helpful,[<class 'str'>],100.00,0.00,0
4,reviews_recommend,[<class 'bool'>],100.00,0.00,0
5,reviews_date,[<class 'str'>],100.00,0.00,0
6,sentiment_analysis,[<class 'int'>],100.00,0.00,0
7,item_id,[<class 'int'>],100.00,0.00,0
8,item_name,[<class 'str'>],100.00,0.00,0
9,playtime_forever,[<class 'int'>],100.00,0.00,0


In [131]:
#minimizo df
df_data = final_df[['release_anio','genres','playtime_forever','user_id','item_id', 'item_name','sentiment_analysis', 'reviews_recommend', 'reviews_date' ]]

In [132]:
utils.types_data_df(df_data)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,release_anio,"[<class 'str'>, <class 'NoneType'>]",99.87,0.13,6347
1,genres,[<class 'str'>],100.00,0.00,0
2,playtime_forever,[<class 'int'>],100.00,0.00,0
3,user_id,[<class 'str'>],100.00,0.00,0
4,item_id,[<class 'int'>],100.00,0.00,0
5,item_name,[<class 'str'>],100.00,0.00,0
6,sentiment_analysis,[<class 'int'>],100.00,0.00,0
7,reviews_recommend,[<class 'bool'>],100.00,0.00,0
8,reviews_date,[<class 'str'>],100.00,0.00,0


In [133]:
df_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5077395 entries, 0 to 5077394
Data columns (total 9 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   release_anio        object
 1   genres              object
 2   playtime_forever    int64 
 3   user_id             object
 4   item_id             int64 
 5   item_name           object
 6   sentiment_analysis  int64 
 7   reviews_recommend   bool  
 8   reviews_date        object
dtypes: bool(1), int64(3), object(5)
memory usage: 314.7+ MB


In [134]:
# Eliminar filas con valores nulos en todas las columnas
final_df_sin_nulos = df_data.dropna()

In [135]:
utils.types_data_df(final_df_sin_nulos)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,release_anio,[<class 'str'>],100.0,0.0,0
1,genres,[<class 'str'>],100.0,0.0,0
2,playtime_forever,[<class 'int'>],100.0,0.0,0
3,user_id,[<class 'str'>],100.0,0.0,0
4,item_id,[<class 'int'>],100.0,0.0,0
5,item_name,[<class 'str'>],100.0,0.0,0
6,sentiment_analysis,[<class 'int'>],100.0,0.0,0
7,reviews_recommend,[<class 'bool'>],100.0,0.0,0
8,reviews_date,[<class 'str'>],100.0,0.0,0


In [136]:
final_df_sin_nulos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5071048 entries, 0 to 5077394
Data columns (total 9 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   release_anio        object
 1   genres              object
 2   playtime_forever    int64 
 3   user_id             object
 4   item_id             int64 
 5   item_name           object
 6   sentiment_analysis  int64 
 7   reviews_recommend   bool  
 8   reviews_date        object
dtypes: bool(1), int64(3), object(5)
memory usage: 353.0+ MB


In [137]:
#Modifico la fecha de reviews_date para extraer el año
final_df_sin_nulos['reviews_date'] = pd.to_datetime(final_df_sin_nulos['reviews_date'], errors='coerce')
final_df_sin_nulos['reviews_anio'] = final_df_sin_nulos['reviews_date'].dt.year

In [138]:
final_df_sin_nulos

,release_anio,genres,playtime_forever,user_id,item_id,item_name,sentiment_analysis,reviews_recommend,reviews_date,reviews_anio
0,2000,Action,6,76561197970982479,10,Counter-Strike,1,True,2011-11-05,2011.0
1,2000,Action,0,js41637,10,Counter-Strike,1,True,2014-06-24,2014.0
2,2000,Action,93,doctr,10,Counter-Strike,2,True,2013-10-14,2013.0
3,2000,Action,0,76561198089393905,10,Counter-Strike,0,True,2015-02-01,2015.0
4,2000,Action,328,WeiEDKrSat,10,Counter-Strike,1,True,2013-12-25,2013.0
...,...,...,...,...,...,...,...,...,...,...
5077390,2016,Indie,0,inven,433920,Aveyond 4: Shadow Of The Mist,1,False,2015-09-09,2015.0
5077391,2016,Rpg,0,inven,433920,Aveyond 4: Shadow Of The Mist,1,False,2015-09-09,2015.0
5077392,2015,Action,226,76561198134165301,352760,Kaiju Panic,1,True,NaT,NaN
5077393,2015,Indie,226,76561198134165301,352760,Kaiju Panic,1,True,NaT,NaN


In [139]:
#Elimino columnna Reviews date
final_df_sin_nulos = final_df_sin_nulos.drop('reviews_date', axis=1)

In [144]:
utils.types_data_df(final_df_sin_nulos)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,release_anio,"[<class 'int'>, <class 'pandas._libs.missing.N...",99.43,0.57,28752
1,genres,[<class 'str'>],100.00,0.00,0
2,playtime_forever,[<class 'int'>],100.00,0.00,0
3,user_id,[<class 'str'>],100.00,0.00,0
4,item_id,[<class 'int'>],100.00,0.00,0
5,item_name,[<class 'str'>],100.00,0.00,0
6,sentiment_analysis,[<class 'int'>],100.00,0.00,0
7,reviews_recommend,[<class 'bool'>],100.00,0.00,0
8,reviews_anio,"[<class 'int'>, <class 'pandas._libs.missing.N...",80.71,19.29,978205


In [145]:
final_df_sin_nulos['release_anio'] = pd.to_numeric(final_df_sin_nulos['release_anio'], errors='coerce').astype('Int64')

In [146]:
final_df_sin_nulos['reviews_anio'] = pd.to_numeric(final_df_sin_nulos['reviews_anio'], errors='coerce').astype('Int64')


In [147]:
utils.types_data_df(final_df_sin_nulos)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,release_anio,"[<class 'int'>, <class 'pandas._libs.missing.N...",99.43,0.57,28752
1,genres,[<class 'str'>],100.00,0.00,0
2,playtime_forever,[<class 'int'>],100.00,0.00,0
3,user_id,[<class 'str'>],100.00,0.00,0
4,item_id,[<class 'int'>],100.00,0.00,0
5,item_name,[<class 'str'>],100.00,0.00,0
6,sentiment_analysis,[<class 'int'>],100.00,0.00,0
7,reviews_recommend,[<class 'bool'>],100.00,0.00,0
8,reviews_anio,"[<class 'int'>, <class 'pandas._libs.missing.N...",80.71,19.29,978205


In [148]:
df_data = final_df_sin_nulos.dropna()

In [149]:
df_data

,release_anio,genres,playtime_forever,user_id,item_id,item_name,sentiment_analysis,reviews_recommend,reviews_anio
0,2000,Action,6,76561197970982479,10,Counter-Strike,1,True,2011
1,2000,Action,0,js41637,10,Counter-Strike,1,True,2014
2,2000,Action,93,doctr,10,Counter-Strike,2,True,2013
3,2000,Action,0,76561198089393905,10,Counter-Strike,0,True,2015
4,2000,Action,328,WeiEDKrSat,10,Counter-Strike,1,True,2013
...,...,...,...,...,...,...,...,...,...
5077387,2015,Indie,251,inven,405960,Sunken,1,False,2015
5077388,2015,Rpg,251,inven,405960,Sunken,1,False,2015
5077389,2016,Adventure,0,inven,433920,Aveyond 4: Shadow Of The Mist,1,False,2015
5077390,2016,Indie,0,inven,433920,Aveyond 4: Shadow Of The Mist,1,False,2015


In [150]:
utils.types_data_df(df_data)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,release_anio,[<class 'int'>],100.0,0.0,0
1,genres,[<class 'str'>],100.0,0.0,0
2,playtime_forever,[<class 'int'>],100.0,0.0,0
3,user_id,[<class 'str'>],100.0,0.0,0
4,item_id,[<class 'int'>],100.0,0.0,0
5,item_name,[<class 'str'>],100.0,0.0,0
6,sentiment_analysis,[<class 'int'>],100.0,0.0,0
7,reviews_recommend,[<class 'bool'>],100.0,0.0,0
8,reviews_anio,[<class 'int'>],100.0,0.0,0


In [151]:
df_data.to_parquet('../data/data_export_api.parquet')

In [154]:
### guardo en compresion brotli
file_path_brotli = '../data/data_export_api_brotli.parquet'
df_data.to_parquet(file_path_brotli, engine='pyarrow', compression='brotli')

In [153]:
file_path_gzip = '../data/data_export_api_gzip.parquet'

# Guardar el DataFrame en un archivo Parquet comprimido con Gzip
df_data.to_parquet(file_path_gzip, engine='pyarrow', compression='gzip')